In [1]:
#cond  124
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy


def inite(elec,orb):
    config=[]
    list1=[]
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
                
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)

                        list1.append(config)
                        config=[]
    return list1

def gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, shots=None, max_iter=100):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    hf_state = qml.qchem.hf_state(active_electrons, qubits)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

    wires=range(qubits)
    params = np.zeros(len(singles) + len(doubles))
    # Define the device
    dev = qml.device("default.qubit", wires=qubits, shots=shots)
    
    # Define the qnode
    if shots==None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                 s_wires=s_wires, d_wires=d_wires,
                                                 hf_state=hf_state)

    gr = circuit(params, wires, s_wires, d_wires, hf_state)
    print('Ground state energy: ', gr)
    return params
def ee_exact(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
    for t in range(1):
        if shots==0:
            #dev = qml.device("default.qubit", wires=qubits)
            dev = qml.device("default.qubit", wires=qubits)
        else:
            #dev = qml.device("default.qubit", wires=qubits,shots=shots)
            dev = qml.device("default.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
            for w in occ:
                qml.X(wires=w)
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #final M matrix
        M = np.zeros((len(list1),len(list1)))
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i == j:
                    M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
        #print("diagonal parts done")
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i!=j:
                    Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                    M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
        #print("off diagonal terms done")
        #ERROR:not subtracting the gs energy
        eig,evec=np.linalg.eig(M)
        values.append(np.sort(eig))
    return values



# Define the molecule
r = 1.88973
symbols  = ['N', 'H', 'H', 'H']
geometry = np.array([[0.0, 0.0,  0.0],
                     [3.7241379310344827*r, 0.0, 0.0],
                     [-0.506*r, 0.876*r, 0.0],
                     [-0.506*r, -0.876*r, 0.0]
                     ], requires_grad = False)
active_electrons = 4
active_orbitals = 4
charge = 0
params = gs_exact(symbols,geometry, active_electrons, active_orbitals, charge,max_iter=50)
#params_exact=params

total_energy = ee_exact(symbols, geometry, active_electrons, active_orbitals, charge,params)
print('exact eigenvalues:\n', total_energy)


Ground state energy:  -55.825048663759084


/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


exact eigenvalues:
 [tensor([-55.82493688, -55.75965086, -55.7594402 , -55.40761347,
        -55.40759085, -55.32484913, -55.31580559, -55.31315373,
        -55.28712741, -55.28273762, -55.26377182, -55.26201155,
        -55.24499796, -55.22277203, -55.18679448, -55.10883783,
        -54.9161245 , -54.85336066, -54.8360962 , -54.81042081,
        -54.73299015, -54.63334281, -54.48266366, -54.45543807,
        -54.40254543, -54.29925994], requires_grad=True)]


In [2]:
def ee(symbols, geometry, active_electrons,  active_orbitals, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis="sto-6g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    list1 = inite(active_electrons,qubits)
    values =[]
    for t in range(10):
        if shots==0:
            #dev = qml.device("lightning.qubit", wires=qubits)
            dev = qml.device("lightning.qubit", wires=qubits)
        else:
            #dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
            for w in occ:
                qml.X(wires=w)
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #final M matrix
        M = np.zeros((len(list1),len(list1)))
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i == j:
                    M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
        #print("diagonal parts done")
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i!=j:
                    Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                    M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
        #print("off diagonal terms done")
        #ERROR:not subtracting the gs energy
        eig,evec=np.linalg.eig(M)
        values.append(np.sort(eig))
    return values

In [3]:
eig500 = ee(symbols, geometry, active_electrons, active_orbitals, charge,params, shots=500)
print('exact eigenvalues:\n', eig500)

/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


exact eigenvalues:
 [tensor([-55.81722916-0.03783403j, -55.81722916+0.03783403j,
        -55.7532567 +0.j        , -55.44276264+0.j        ,
        -55.3984351 +0.j        , -55.36394445+0.j        ,
        -55.30643598-0.00669521j, -55.30643598+0.00669521j,
        -55.2769167 +0.j        , -55.2655804 +0.j        ,
        -55.25268137-0.04087066j, -55.25268137+0.04087066j,
        -55.23353567+0.j        , -55.20406726-0.02831824j,
        -55.20406726+0.02831824j, -55.07399639+0.j        ,
        -54.93382022+0.j        , -54.86806861+0.j        ,
        -54.8531116 +0.j        , -54.84195968+0.j        ,
        -54.75259796+0.j        , -54.59732043+0.j        ,
        -54.46896583-0.02650961j, -54.46896583+0.02650961j,
        -54.33934684-0.03379576j, -54.33934684+0.03379576j], requires_grad=True), tensor([-55.84922499+0.j        , -55.78966033+0.j        ,
        -55.70833198+0.j        , -55.42718901+0.j        ,
        -55.38735539+0.j        , -55.35157283+0.j       

In [4]:
eig1k = ee(symbols, geometry, active_electrons, active_orbitals, charge,params, shots=1000)
print('exact eigenvalues:\n', eig1k)

exact eigenvalues:
 [tensor([-55.83565914+0.j        , -55.77029181+0.j        ,
        -55.73648104+0.j        , -55.41318086+0.j        ,
        -55.39802527+0.j        , -55.322103  -0.01598516j,
        -55.322103  +0.01598516j, -55.32140607-0.00805114j,
        -55.32140607+0.00805114j, -55.26798464+0.j        ,
        -55.25848947-0.01553794j, -55.25848947+0.01553794j,
        -55.2199129 -0.00986009j, -55.2199129 +0.00986009j,
        -55.15875694+0.j        , -55.08257263+0.j        ,
        -54.88982736+0.j        , -54.85619284+0.j        ,
        -54.81645713-0.0081094j , -54.81645713+0.0081094j ,
        -54.75467544+0.j        , -54.6348757 +0.j        ,
        -54.47502739+0.j        , -54.42153331+0.j        ,
        -54.37955687+0.j        , -54.30480934+0.j        ], requires_grad=True), tensor([-55.82652212+0.j        , -55.7824167 +0.j        ,
        -55.75108912+0.j        , -55.4859439 +0.j        ,
        -55.43143837+0.j        , -55.35401362+0.j       

In [5]:
eig10k = ee(symbols, geometry, active_electrons, active_orbitals, charge,params, shots=10000)
print('exact eigenvalues:\n', eig10k)

exact eigenvalues:
 [tensor([-55.82346031+0.j        , -55.76678529+0.j        ,
        -55.75630761+0.j        , -55.42087989+0.j        ,
        -55.4034474 +0.j        , -55.32156165+0.j        ,
        -55.31259181+0.j        , -55.30281253+0.j        ,
        -55.28881517+0.j        , -55.28037956+0.j        ,
        -55.26178842-0.00308237j, -55.26178842+0.00308237j,
        -55.24328321+0.j        , -55.22836446+0.j        ,
        -55.18652062+0.j        , -55.10375081+0.j        ,
        -54.91779619+0.j        , -54.8455482 -0.00360455j,
        -54.8455482 +0.00360455j, -54.80418655+0.j        ,
        -54.72874801+0.j        , -54.63141524+0.j        ,
        -54.48236586+0.j        , -54.44928829+0.j        ,
        -54.41064783+0.j        , -54.28820411+0.j        ], requires_grad=True), tensor([-55.82797895+0.j        , -55.76721649+0.j        ,
        -55.74977894+0.j        , -55.40564451-0.0024958j ,
        -55.40564451+0.0024958j , -55.33030199+0.j       

## Graph

In [ ]:
import matplotlib.pyplot as plt
# Exact eigenvalues
exact_1_ex = total_energy[0][0]
exact_2_ex = total_energy[0][1]
exact_3_ex = total_energy[0][2]
exact_4_ex = total_energy[0][3]
exact_5_ex = total_energy[0][4]
exact_6_ex = total_energy[0][5]
exact_7_ex = total_energy[0][6]
exact_8_ex = total_energy[0][7]
exact_9_ex = total_energy[0][8]
exact_10_ex = total_energy[0][9]





# Shot counts
shots = [500, 1000, 10000] 
print(exact_1_ex)
print(exact_2_ex)

In [ ]:
ex1 = np.array([means_500[0].real, means_1k[0].real, means_10k[0].real])
ex2 = np.array([means_500[1].real, means_1k[1].real, means_10k[1].real])
ex3 = np.array([means_500[2].real, means_1k[2].real, means_10k[2].real])
ex4 = np.array([means_500[3].real, means_1k[3].real, means_10k[3].real])
ex5 = np.array([means_500[4].real, means_1k[4].real, means_10k[4].real])
ex6 = np.array([means_500[5].real, means_1k[5].real, means_10k[5].real])
ex7 = np.array([means_500[6].real, means_1k[6].real, means_10k[6].real])
ex8 = np.array([means_500[7].real, means_1k[7].real, means_10k[7].real])
ex9 = np.array([means_500[8].real, means_1k[8].real, means_10k[8].real])
ex10 = np.array([means_500[9].real, means_1k[9].real, means_10k[9].real])
ex11 = np.array([ means_500[10].real, means_1k[10].real, means_10k[10].real])
ex12 = np.array([ means_500[11].real, means_1k[11].real, means_10k[11].real])
ex13 = np.array([ means_500[12].real, means_1k[12].real, means_10k[12].real])
ex14 = np.array([ means_500[13].real, means_1k[13].real, means_10k[13].real])
ex15 = np.array([ means_500[14].real, means_1k[14].real, means_10k[14].real])
ex16 = np.array([ means_500[15].real, means_1k[15].real, means_10k[15].real])
ex17 = np.array([ means_500[16].real, means_1k[16].real, means_10k[16].real])
ex18 = np.array([ means_500[17].real, means_1k[17].real, means_10k[17].real])
ex19 = np.array([ means_500[18].real, means_1k[18].real, means_10k[18].real])
ex20 = np.array([ means_500[19].real, means_1k[19].real, means_10k[19].real])
ex21 = np.array([ means_500[20].real, means_1k[20].real, means_10k[20].real])


# variance compiled
var_ex1 = [val_500[0], val_1k[0], val_10k[0]]
var_ex2 = [val_500[1], val_1k[1], val_10k[1]]
var_ex3 = [val_500[2], val_1k[2], val_10k[2]]
var_ex4 = [val_500[3], val_1k[3], val_10k[3]]
var_ex5 = [val_500[4], val_1k[4], val_10k[4]]
var_ex6 = [abs(val_500[5]), abs(val_1k[5]), abs(val_10k[5])]
var_ex7 = [abs(val_500[6]), abs(val_1k[6]), abs(val_10k[6])]
var_ex8 = [abs(val_500[7]), abs(val_1k[7]), abs(val_10k[7])]
var_ex9 = [[abs(val_500[8]), abs(val_1k[8]), abs(val_10k[8])]]
var_ex10 = [abs(val_500[9]), abs(val_1k[9]), abs(val_10k[9])]
var_ex11 = [ val_500[10], val_1k[10], val_10k[10]]
var_ex12 = [ val_500[11], val_1k[11], val_10k[11]]
var_ex13 = [ val_500[12], val_1k[12], val_10k[12]]
var_ex14 = [ val_500[13], val_1k[13], val_10k[13]]
var_ex15 = [ val_500[14], val_1k[14], val_10k[14]]
var_ex16 = [ val_500[15], val_1k[15], val_10k[15]]
var_ex17 = [ val_500[16], val_1k[16], val_10k[16]]
var_ex18 = [ val_500[17], val_1k[17], val_10k[17]]
var_ex19 = [ val_500[18], val_1k[18], val_10k[18]]
var_ex20 = [ val_500[19], val_1k[19], val_10k[19]]
var_ex21 = [ val_500[20], val_1k[20], val_10k[20]]
var_ex22 = [ val_500[21], val_1k[21], val_10k[21]]
var_ex23 = [ val_500[22], val_1k[22], val_10k[22]]
var_ex24 = [ val_500[23], val_1k[23], val_10k[23]]


In [ ]:
# Scale factor for error bars 
scale_factor = 3
# Create a figure
fig, ax1 = plt.subplots(figsize=(10, 5))

# Plot exact values as horizontal lines
ax1.axhline(y=exact_1_ex, color='r', linestyle='-', label='Exact energy')
ax1.axhline(y=exact_2_ex, color='b', linestyle='-')
ax1.axhline(y=exact_3_ex, color='y', linestyle='-')
ax1.axhline(y=exact_4_ex, color='#EE82EE', linestyle='-')
ax1.axhline(y=exact_5_ex, color='black', linestyle='-')
ax1.axhline(y=exact_6_ex, color='g', linestyle='-')
ax1.axhline(y=exact_7_ex, color='brown', linestyle='-')
ax1.axhline(y=exact_8_ex, color='b', linestyle='-')
ax1.axhline(y=exact_9_ex, color='violet', linestyle='-')
ax1.axhline(y=exact_10_ex, color='black', linestyle='-')



# Plot predicted values as dots with scaled RMSD as error bars
ax1.errorbar(shots, ex1, yerr=var_ex1, fmt='o', color='r', label='Measured energy', capsize=5, markersize=5)
ax1.errorbar(shots, ex2, yerr=var_ex2, fmt='o', color='b', capsize=5, markersize=5)
ax1.errorbar(shots, ex3, yerr=var_ex3, fmt='o', color='y', capsize=5, markersize=5)
ax1.errorbar(shots, ex4, yerr=var_ex4, fmt='o', color='violet', capsize=5, markersize=5)
ax1.errorbar(shots, ex5, yerr=var_ex5, fmt='o', color='black', capsize=5, markersize=5)
ax1.errorbar(shots, ex6, yerr=var_ex6, fmt='o', color='g', capsize=5, markersize=5)
ax1.errorbar(shots, ex7, yerr=var_ex7, fmt='o', color='brown', capsize=5, markersize=5)
ax1.errorbar(shots, ex8, yerr=var_ex8, fmt='o', color='b', capsize=5, markersize=5)
ax1.errorbar(shots, ex9, yerr=var_ex9, fmt='o', color='violet', capsize=5, markersize=5)
ax1.errorbar(shots, ex10, yerr=var_ex10, fmt='o', color='black', capsize=5, markersize=5)



# Set y-axis limit
#ax1.set_ylim(-2.5, -1.1)
plt.xlabel('Shot Count')
ax1.set_ylabel('Energy in hartree')
ax1.set_title('qscEOM NH3 (4,4)Deviation Between Exact Energy and Measured Energy with Variance')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fontsize='small', ncol=4)
#plt.grid(True)
plt.xscale('log')  
plt.show()

